# Exemple de soumission valide pour le challenge

Ce notebook sert de point de départ pour avancer dans la compétition : nous ne présenterons pas ici les meilleurs pratiques ou le meilleur algorithme pour remporter la compétition.Commençons par importer les données.

In [14]:
import numpy as np
import pandas as pd

df = pd.read_csv("train.csv")
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
df = df.sort_values(by="date")
df.head()

,ticker,commodity,date,open,high,low,volume,ID,target
0,CL=F,Crude Oil,2000-08-23,31.950001,32.799999,31.950001,79385,0,31.629999
1,CL=F,Crude Oil,2000-08-24,31.900000,32.240002,31.400000,72978,1,32.049999
2,CL=F,Crude Oil,2000-08-25,31.700001,32.099998,31.320000,44601,2,32.869999
3,CL=F,Crude Oil,2000-08-28,32.040001,32.919998,31.860001,46770,3,32.720001
4,CL=F,Crude Oil,2000-08-29,32.820000,33.029999,32.560001,49131,4,33.400002


Il faut rendre les données exploitable pour un algorithme de Machine Learning. Nous allons couper la base d'entraînement en un jeu d'entraînement et de validation. Pour le faire, gardons en tête que nous devons conserver l'ordre temporel : il n'est pas le même pour toute les commodité :

In [15]:
for ticker in df["ticker"].value_counts().index:
    temp = df.loc[df["ticker"] == ticker, ]
    print(ticker, ":", np.min(temp["date"]), np.max(temp["date"]))

CL=F : 2000-08-23 00:00:00 2019-08-29 00:00:00
NG=F : 2000-08-30 00:00:00 2019-08-29 00:00:00
HO=F : 2000-09-01 00:00:00 2019-08-30 00:00:00
RB=F : 2000-11-01 00:00:00 2019-09-12 00:00:00
BZ=F : 2007-07-30 00:00:00 2021-01-28 00:00:00


Nous allons donc construire le jeu d'entraînement avec les valorisations jusqu'à fin 2017 et nous utiliserons le reste comme jeu de validation.

In [16]:
train, validation = df.loc[df["date"] < "2018-01-01", ], df.loc[df["date"] >= "2018-01-01", ]
X_train = train.drop(columns=["ticker", "commodity", "date", "ID", "target"])
X_valid = validation.drop(columns=["ticker", "commodity", "date", "ID", "target"])
y_train = train["target"]
y_valid = validation["target"]

Nous pouvons maintenant entraîner et mesurer la performance d'un algorithme.

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression().fit(X_train, y_train)
y_pred = model.predict(X_valid)

RMSE = lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true=y_true, y_pred=y_pred))
performance = RMSE(y_valid, y_pred)
print(f"RMSE: {performance:.4}")


RMSE: 0.9557


Nous avons un modèle fonctionnel. Passons au test ! Pour qu'une soumission soit valide, il faut que l'on ait la colonne *ID* et une colonne nommée *predicted* qui contiendra la valeur prédite par l'algorithme. 

In [18]:
test = pd.read_csv("test.csv")
X_test = test.drop(columns=["ticker", "commodity", "date", "ID"])
y_pred = model.predict(X_test)

submission = pd.DataFrame({
    "ID": test["ID"],
    "predicted": y_pred
})
submission.to_csv("submission.csv", index=False)

Il ne reste plus qu'à charger le fichier sur Kaggle !